**Importing library**

In [ ]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=c7696d89d33c724d864c93a799a799c5eb1f73846902898ee68d0421a5f817d6
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName("DatasetAnalysis").getOrCreate()

**creation of dataframe**

In [ ]:
df= spark.read.csv("Electric_Vehicle_Population_Data.csv", header=True, inferSchema=True)

In [ ]:
df.show(15)

+----------+---------+------------+-----+-----------+----------+---------+--------+---------------------+-------------------------------------------------+--------------+---------+--------------------+--------------+--------------------+--------------------+-----------------+
|VIN (1-10)|   County|        City|State|Postal Code|Model Year|     Make|   Model|Electric Vehicle Type|Clean Alternative Fuel Vehicle (CAFV) Eligibility|Electric Range|Base MSRP|Legislative District|DOL Vehicle ID|    Vehicle Location|    Electric Utility|2020 Census Tract|
+----------+---------+------------+-----+-----------+----------+---------+--------+---------------------+-------------------------------------------------+--------------+---------+--------------------+--------------+--------------------+--------------------+-----------------+
|1N4AZ0CP5D|   Kitsap|   Bremerton|   WA|      98310|      2013|   NISSAN|    LEAF| Battery Electric ...|                             Clean Alternative...|            75

In [ ]:
df.count()

24718

there are 138779 records in dataset

**check schema**

*   check the datatype of columns in dataset



In [ ]:
df.printSchema()

root
 |-- VIN (1-10): string (nullable = true)
 |-- County: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Model Year: integer (nullable = true)
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Electric Vehicle Type: string (nullable = true)
 |-- Clean Alternative Fuel Vehicle (CAFV) Eligibility: string (nullable = true)
 |-- Electric Range: integer (nullable = true)
 |-- Base MSRP: integer (nullable = true)
 |-- Legislative District: integer (nullable = true)
 |-- DOL Vehicle ID: integer (nullable = true)
 |-- Vehicle Location: string (nullable = true)
 |-- Electric Utility: string (nullable = true)
 |-- 2020 Census Tract: long (nullable = true)



In [ ]:
df.describe().show()

+-------+----------+------+--------+-----+------------------+------------------+-----+--------------------+---------------------+-------------------------------------------------+-----------------+------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|summary|VIN (1-10)|County|    City|State|       Postal Code|        Model Year| Make|               Model|Electric Vehicle Type|Clean Alternative Fuel Vehicle (CAFV) Eligibility|   Electric Range|         Base MSRP|Legislative District|     DOL Vehicle ID|    Vehicle Location|    Electric Utility|   2020 Census Tract|
+-------+----------+------+--------+-----+------------------+------------------+-----+--------------------+---------------------+-------------------------------------------------+-----------------+------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|  count|     24718| 24715|   24715|2

**Creation of View**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQLExample").getOrCreate()
data = spark.read.csv("Electric_Vehicle_Population_Data.csv", header=True, inferSchema=True)
# Create a view for DataFrame
data.createOrReplaceTempView("vdata")



## Analysing through  SQL Queries

In [ ]:

result = spark.sql("SELECT * FROM vdata")
result.show()
spark.stop()


+----------+---------+------------+-----+-----------+----------+---------+--------+---------------------+-------------------------------------------------+--------------+---------+--------------------+--------------+--------------------+--------------------+-----------------+
|VIN (1-10)|   County|        City|State|Postal Code|Model Year|     Make|   Model|Electric Vehicle Type|Clean Alternative Fuel Vehicle (CAFV) Eligibility|Electric Range|Base MSRP|Legislative District|DOL Vehicle ID|    Vehicle Location|    Electric Utility|2020 Census Tract|
+----------+---------+------------+-----+-----------+----------+---------+--------+---------------------+-------------------------------------------------+--------------+---------+--------------------+--------------+--------------------+--------------------+-----------------+
|1N4AZ0CP5D|   Kitsap|   Bremerton|   WA|      98310|      2013|   NISSAN|    LEAF| Battery Electric ...|                             Clean Alternative...|            75

**Most Sold EV car**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQLExample").getOrCreate()
data = spark.read.csv("Electric_Vehicle_Population_Data.csv", header=True, inferSchema=True)
data.createOrReplaceTempView("vdata")
result1 = spark.sql("SELECT model,count(model) FROM vdata group by 1 order by 2 desc limit 10;")
result1.show()
spark.stop()


+-----------+------------+
|      model|count(model)|
+-----------+------------+
|    MODEL 3|       25925|
|    MODEL Y|       24859|
|       LEAF|       13030|
|    MODEL S|        7442|
|    BOLT EV|        5467|
|    MODEL X|        4914|
|       VOLT|        4871|
|       NIRO|        2677|
|       ID.4|        2656|
|PRIUS PRIME|        2473|
+-----------+------------+



**Highest mileage Ev model**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQLExample").getOrCreate()
data = spark.read.csv("Electric_Vehicle_Population_Data.csv", header=True, inferSchema=True)
data.createOrReplaceTempView("vdata")
result2 = spark.sql("SELECT distinct make,model,`electric range` AS Range FROM vdata order by 3 desc limit 5")
result2.show()
spark.stop()

+-----+-------+-----+
| make|  model|Range|
+-----+-------+-----+
|TESLA|MODEL S|  337|
|TESLA|MODEL S|  330|
|TESLA|MODEL 3|  322|
|TESLA|MODEL 3|  308|
|TESLA|MODEL X|  293|
+-----+-------+-----+



**Most and least used electric car cities.**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQLExample").getOrCreate()
data = spark.read.csv("Electric_Vehicle_Population_Data.csv", header=True, inferSchema=True)
data.createOrReplaceTempView("vdata")
# Query for the most used electric car cities
most_used_cities = spark.sql("""
    SELECT city, COUNT(*) AS carCount
    FROM vdata
    GROUP BY city
    ORDER BY carCount DESC
    LIMIT 1
""")
most_used_cities.show()
# Query for the least used electric car cities
least_used_cities = spark.sql("""
    SELECT city, COUNT(*) AS carCount
    FROM vdata
    GROUP BY city
    ORDER BY carCount ASC
    LIMIT 1
""")
least_used_cities.show()
spark.stop()


+-------+--------+
|   city|carCount|
+-------+--------+
|Seattle|   23964|
+-------+--------+

+---------+--------+
|     city|carCount|
+---------+--------+
|Fairbanks|       1|
+---------+--------+



**Type of electric vechiles and their count over the years**

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SQLExample").getOrCreate()
data = spark.read.csv("Electric_Vehicle_Population_Data.csv", header=True, inferSchema=True)
data.createOrReplaceTempView("vdata")

# Query for the vehicle types and their counts over the years
ev_type_counts = spark.sql("""
    SELECT `Model Year`, `Electric Vehicle Type`, COUNT(*) AS Count
    FROM vdata
    GROUP BY `Model Year`, `Electric Vehicle Type`
    ORDER BY `Model Year` DESC, Count DESC
    LIMIT 20
""")
ev_type_counts.show()
spark.stop()


+----------+---------------------+-----+
|Model Year|Electric Vehicle Type|Count|
+----------+---------------------+-----+
|      2024| Plug-in Hybrid El...|   74|
|      2024| Battery Electric ...|   50|
|      2023| Battery Electric ...|22411|
|      2023| Plug-in Hybrid El...| 4575|
|      2022| Battery Electric ...|23476|
|      2022| Plug-in Hybrid El...| 4221|
|      2021| Battery Electric ...|14658|
|      2021| Plug-in Hybrid El...| 3600|
|      2020| Battery Electric ...| 9387|
|      2020| Plug-in Hybrid El...| 1669|
|      2019| Battery Electric ...| 8595|
|      2019| Plug-in Hybrid El...| 1911|
|      2018| Battery Electric ...|10018|
|      2018| Plug-in Hybrid El...| 4351|
|      2017| Battery Electric ...| 4450|
|      2017| Plug-in Hybrid El...| 4105|
|      2016| Battery Electric ...| 3875|
|      2016| Plug-in Hybrid El...| 1783|
|      2015| Battery Electric ...| 3607|
|      2015| Plug-in Hybrid El...| 1314|
+----------+---------------------+-----+



**Data Visualization**

*   graphs are plotted with the help of tableau
*   Few insights and hidden patterns are explored through this graphs



**Linear regression model**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Create a Spark session
spark = SparkSession.builder.appName("LinearRegressionExample").getOrCreate()

# Load and preprocess data
data = spark.read.csv("Electric_Vehicle_Population_Data.csv", header=True, inferSchema=True)
# Preprocess data: handle missing values, encode categorical variables, etc.

# Select features and target variable
selected_features = ["Model Year", "Make", "Model", "Electric Vehicle Type"]
target_variable = "Electric Range"

# StringIndexer for categorical features
indexers = [
    StringIndexer(inputCol=feature, outputCol=f"{feature}_index", handleInvalid="keep")
    for feature in selected_features
]

# Apply StringIndexer transformations
indexer_models = [indexer.fit(data) for indexer in indexers]
data_indexed = data
for indexer_model in indexer_models:
    data_indexed = indexer_model.transform(data_indexed)

# Assemble features into a feature vector
assembler = VectorAssembler(inputCols=[f"{feature}_index" for feature in selected_features], outputCol="features")
data_assembled = assembler.transform(data_indexed)

# Split data into training and testing sets
train_data, test_data = data_assembled.randomSplit([0.8, 0.2], seed=123)

# Train the Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol=target_variable)
model = lr.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Evaluate the model's performance
evaluator = RegressionEvaluator(labelCol=target_variable, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

# Print RMSE
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Stop the Spark session
spark.stop()


Root Mean Squared Error (RMSE): 82.536094698762
